# OCTDL Eye Disease Classification
**Deep Learning Model Comparison: ResNet50 vs InceptionV3**

Advanced Machine Learning Final Project | June 2025

## 1. Setup & Imports

In [ ]:
# Core imports
import os, shutil, random, json, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# TensorFlow & Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet50, InceptionV3
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess

# Configuration
tf.random.set_seed(123)
print(f"TensorFlow: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

## 2. Configuration & Data Setup

In [ ]:
# Project configuration
PROJECT_DIR = r'c:\Users\Mr Computer\Desktop\Folders\CAP\S#3\T#2\Advance Machine Learning\Final_Project'
BASE_DATA_DIR = os.path.join(PROJECT_DIR, 'archive', 'OCTDL', 'OCTDL')

# Training parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123
INITIAL_EPOCHS = 15
FINE_TUNE_EPOCHS = 15

# Data directories
TRAIN_DIR = os.path.join(PROJECT_DIR, 'train_data')
VAL_DIR = os.path.join(PROJECT_DIR, 'val_data')
TEST_DIR = os.path.join(PROJECT_DIR, 'test_data')

print(f"Base data directory: {BASE_DATA_DIR}")
print(f"Data exists: {os.path.exists(BASE_DATA_DIR)}")

In [ ]:
# Create train/val/test splits
def create_data_splits():
    # Remove existing splits
    for folder in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
        if os.path.exists(folder):
            shutil.rmtree(folder)
        os.makedirs(folder, exist_ok=True)
    
    # Split function
    def split_class_data(src_dir, dest_dirs, ratios=[0.8, 0.1, 0.1]):
        for class_name in os.listdir(src_dir):
            class_path = os.path.join(src_dir, class_name)
            if not os.path.isdir(class_path):
                continue
            
            images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.png', '.jpeg', '.tiff'))]
            random.seed(SEED)
            random.shuffle(images)
            
            total = len(images)
            if total == 0:
                continue
            
            # Calculate splits
            train_end = int(total * ratios[0])
            val_end = int(total * (ratios[0] + ratios[1]))
            
            splits = {
                dest_dirs[0]: images[:train_end],
                dest_dirs[1]: images[train_end:val_end],
                dest_dirs[2]: images[val_end:]
            }
            
            # Copy files
            for dest_dir, file_list in splits.items():
                class_dest = os.path.join(dest_dir, class_name)
                os.makedirs(class_dest, exist_ok=True)
                
                for fname in file_list:
                    src_file = os.path.join(class_path, fname)
                    dst_file = os.path.join(class_dest, fname)
                    shutil.copy2(src_file, dst_file)
    
    # Create splits
    split_class_data(BASE_DATA_DIR, [TRAIN_DIR, VAL_DIR, TEST_DIR])
    
    # Count images
    def count_images(directory):
        count = 0
        for root, dirs, files in os.walk(directory):
            count += len([f for f in files if f.lower().endswith(('.jpg', '.png', '.jpeg', '.tiff'))])
        return count
    
    train_count = count_images(TRAIN_DIR)
    val_count = count_images(VAL_DIR)
    test_count = count_images(TEST_DIR)
    
    print(f"Train: {train_count}, Val: {val_count}, Test: {test_count}")
    return train_count > 0

# Create data splits
data_ready = create_data_splits()
print(f"Data preparation: {'✅ Success' if data_ready else '❌ Failed'}")

## 3. Data Loading & Augmentation

In [ ]:
# Create datasets
train_ds = image_dataset_from_directory(
    TRAIN_DIR, labels='inferred', label_mode='categorical',
    batch_size=BATCH_SIZE, image_size=IMG_SIZE, shuffle=True, seed=SEED
)

val_ds = image_dataset_from_directory(
    VAL_DIR, labels='inferred', label_mode='categorical',
    batch_size=BATCH_SIZE, image_size=IMG_SIZE, shuffle=False
)

test_ds = image_dataset_from_directory(
    TEST_DIR, labels='inferred', label_mode='categorical',
    batch_size=BATCH_SIZE, image_size=IMG_SIZE, shuffle=False
)

# Get class names and optimize datasets
class_names = train_ds.class_names
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

print(f"Classes: {class_names}")
print(f"Number of classes: {len(class_names)}")

In [ ]:
# Data augmentation for medical images
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15, fill_mode='constant', fill_value=0.0),
    layers.RandomZoom(0.15, fill_mode='constant', fill_value=0.0),
    layers.RandomTranslation(0.1, 0.1, fill_mode='constant', fill_value=0.0),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.15),
    layers.GaussianNoise(0.01),
], name="data_augmentation")

# Calculate class weights for imbalanced data
def calculate_class_weights(dataset, class_names):
    class_counts = {name: 0 for name in class_names}
    
    for images, labels in dataset:
        for label in labels:
            class_idx = tf.argmax(label).numpy()
            class_counts[class_names[class_idx]] += 1
    
    total_samples = sum(class_counts.values())
    class_weights = {}
    
    for i, class_name in enumerate(class_names):
        class_weights[i] = total_samples / (len(class_names) * class_counts[class_name])
    
    return class_weights, class_counts

class_weights, train_class_counts = calculate_class_weights(train_ds, class_names)

print("Class distribution:")
for name, count in train_class_counts.items():
    print(f"  {name}: {count}")

print("\nClass weights:")
for i, weight in class_weights.items():
    print(f"  {class_names[i]}: {weight:.2f}")

## 4. Model Architecture

In [ ]:
# Model creation function
def create_model(architecture='resnet50', num_classes=None, input_shape=None):
    if num_classes is None:
        num_classes = len(class_names)
    if input_shape is None:
        input_shape = IMG_SIZE + (3,)
    
    # Load base model
    if architecture.lower() == 'resnet50':
        base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
        preprocessing_func = resnet_preprocess
        model_name = "ResNet50_OCT_Classifier"
    elif architecture.lower() == 'inceptionv3':
        base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=input_shape)
        preprocessing_func = inception_preprocess
        model_name = "InceptionV3_OCT_Classifier"
    else:
        raise ValueError("Architecture must be 'resnet50' or 'inceptionv3'")
    
    # Freeze base model
    base_model.trainable = False
    
    # Build model
    inputs = keras.Input(shape=input_shape)
    x = preprocessing_func(inputs)
    x = data_augmentation(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name=model_name)
    
    # Compile model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.F1Score()]
    )
    
    return model, base_model

# Create both models
print("Creating models...")
resnet_model, resnet_base = create_model('resnet50')
inception_model, inception_base = create_model('inceptionv3')

print(f"ResNet50 parameters: {resnet_model.count_params():,}")
print(f"InceptionV3 parameters: {inception_model.count_params():,}")

## 5. Training Setup

In [ ]:
# Create callbacks
def create_callbacks(model_name, project_dir):
    checkpoint_path = os.path.join(project_dir, f"best_model_{model_name.lower()}.keras")
    
    callbacks_list = [
        callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1),
        callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1),
        callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6, verbose=1),
        callbacks.TensorBoard(log_dir=os.path.join(project_dir, "logs", model_name.lower()), histogram_freq=1)
    ]
    
    return callbacks_list, checkpoint_path

# Create callbacks for both models
resnet_callbacks, resnet_checkpoint = create_callbacks("ResNet50", PROJECT_DIR)
inception_callbacks, inception_checkpoint = create_callbacks("InceptionV3", PROJECT_DIR)

print("Training callbacks configured")

## 6. Model Training

In [ ]:
# Training function
def train_model(model, base_model, model_name, callbacks_list, train_dataset, val_dataset, class_weights):
    print(f"\n{'='*50}")
    print(f"Training {model_name}")
    print(f"{'='*50}")
    
    # Phase 1: Frozen backbone
    print(f"\nPhase 1: Initial training ({INITIAL_EPOCHS} epochs)")
    history_initial = model.fit(
        train_dataset, epochs=INITIAL_EPOCHS, validation_data=val_dataset,
        callbacks=callbacks_list, class_weight=class_weights, verbose=1
    )
    
    # Phase 2: Fine-tuning
    print(f"\nPhase 2: Fine-tuning ({FINE_TUNE_EPOCHS} epochs)")
    base_model.trainable = True
    
    # Unfreeze top layers
    if model_name.lower().startswith('resnet'):
        for layer in base_model.layers[:-10]:
            layer.trainable = False
    else:  # InceptionV3
        for layer in base_model.layers[:-20]:
            layer.trainable = False
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=optimizers.Adam(learning_rate=1e-5),
        loss="categorical_crossentropy",
        metrics=["accuracy", keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.F1Score()]
    )
    
    total_epochs = INITIAL_EPOCHS + FINE_TUNE_EPOCHS
    history_finetune = model.fit(
        train_dataset, epochs=total_epochs, initial_epoch=INITIAL_EPOCHS,
        validation_data=val_dataset, callbacks=callbacks_list,
        class_weight=class_weights, verbose=1
    )
    
    print(f"\n{model_name} training completed!")
    
    return {
        'initial_history': history_initial,
        'finetune_history': history_finetune,
        'model': model,
        'model_name': model_name
    }

# Train both models
print("Starting model training...")

resnet_results = train_model(
    resnet_model, resnet_base, "ResNet50", resnet_callbacks, train_ds, val_ds, class_weights
)

inception_results = train_model(
    inception_model, inception_base, "InceptionV3", inception_callbacks, train_ds, val_ds, class_weights
)

## 7. Model Evaluation

In [1]:
# Comprehensive model evaluation
def evaluate_models(resnet_model, inception_model, test_dataset, class_names):
    print("\n" + "="*60)
    print("MODEL EVALUATION")
    print("="*60)
    
    # Evaluate models
    print("\nEvaluating ResNet50...")
    resnet_results = resnet_model.evaluate(test_dataset, verbose=1)
    resnet_metrics = dict(zip(resnet_model.metrics_names, resnet_results))
    
    print("\nEvaluating InceptionV3...")
    inception_results = inception_model.evaluate(test_dataset, verbose=1)
    inception_metrics = dict(zip(inception_model.metrics_names, inception_results))
    
    # Generate predictions
    print("\nGenerating predictions...")
    resnet_predictions = resnet_model.predict(test_dataset, verbose=1)
    inception_predictions = inception_model.predict(test_dataset, verbose=1)
    
    # Get true labels
    y_true = []
    for _, labels in test_dataset:
        y_true.extend(labels.numpy())
    y_true = np.array(y_true)
    
    # Convert to class indices
    resnet_pred_classes = np.argmax(resnet_predictions, axis=1)
    inception_pred_classes = np.argmax(inception_predictions, axis=1)
    y_true_classes = np.argmax(y_true, axis=1)
    
    return {
        'resnet_metrics': resnet_metrics,
        'inception_metrics': inception_metrics,
        'resnet_predictions': resnet_predictions,
        'inception_predictions': inception_predictions,
        'resnet_pred_classes': resnet_pred_classes,
        'inception_pred_classes': inception_pred_classes,
        'y_true': y_true,
        'y_true_classes': y_true_classes
    }

# Evaluate models
eval_results = evaluate_models(resnet_model, inception_model, test_ds, class_names)

NameError: name 'resnet_model' is not defined

## 8. Results Visualization

In [2]:
# Training history comparison
def plot_training_comparison(resnet_results, inception_results):
    # Combine histories
    def combine_history(initial, finetune):
        combined = {}
        for key in initial.history.keys():
            combined[key] = initial.history[key] + finetune.history[key]
        return combined
    
    resnet_hist = combine_history(resnet_results['initial_history'], resnet_results['finetune_history'])
    inception_hist = combine_history(inception_results['initial_history'], inception_results['finetune_history'])
    
    epochs = range(len(resnet_hist['accuracy']))
    
    plt.figure(figsize=(15, 5))
    
    # Accuracy
    plt.subplot(1, 3, 1)
    plt.plot(epochs, resnet_hist['accuracy'], 'b-', label='ResNet50 Train')
    plt.plot(epochs, resnet_hist['val_accuracy'], 'b--', label='ResNet50 Val')
    plt.plot(epochs, inception_hist['accuracy'], 'r-', label='InceptionV3 Train')
    plt.plot(epochs, inception_hist['val_accuracy'], 'r--', label='InceptionV3 Val')
    plt.title('Accuracy Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Loss
    plt.subplot(1, 3, 2)
    plt.plot(epochs, resnet_hist['loss'], 'b-', label='ResNet50 Train')
    plt.plot(epochs, resnet_hist['val_loss'], 'b--', label='ResNet50 Val')
    plt.plot(epochs, inception_hist['loss'], 'r-', label='InceptionV3 Train')
    plt.plot(epochs, inception_hist['val_loss'], 'r--', label='InceptionV3 Val')
    plt.title('Loss Comparison')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Final metrics comparison
    plt.subplot(1, 3, 3)
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    resnet_final = [resnet_hist['val_accuracy'][-1], resnet_hist['val_precision'][-1], 
                    resnet_hist['val_recall'][-1], resnet_hist['val_f1_score'][-1]]
    inception_final = [inception_hist['val_accuracy'][-1], inception_hist['val_precision'][-1], 
                       inception_hist['val_recall'][-1], inception_hist['val_f1_score'][-1]]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.bar(x - width/2, resnet_final, width, label='ResNet50', alpha=0.8)
    plt.bar(x + width/2, inception_final, width, label='InceptionV3', alpha=0.8)
    
    plt.title('Final Validation Metrics')
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.xticks(x, metrics)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot comparison
plot_training_comparison(resnet_results, inception_results)

NameError: name 'resnet_results' is not defined

In [ ]:
# Confusion matrices
from sklearn.metrics import confusion_matrix, classification_report

def plot_confusion_matrices(eval_results, class_names):
    # Calculate confusion matrices
    resnet_cm = confusion_matrix(eval_results['y_true_classes'], eval_results['resnet_pred_classes'])
    inception_cm = confusion_matrix(eval_results['y_true_classes'], eval_results['inception_pred_classes'])
    
    # Normalize
    resnet_cm_norm = resnet_cm.astype('float') / resnet_cm.sum(axis=1)[:, np.newaxis]
    inception_cm_norm = inception_cm.astype('float') / inception_cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(12, 5))
    
    # ResNet50
    plt.subplot(1, 2, 1)
    sns.heatmap(resnet_cm_norm, annot=True, fmt='.2f', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title('ResNet50 - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    
    # InceptionV3
    plt.subplot(1, 2, 2)
    sns.heatmap(inception_cm_norm, annot=True, fmt='.2f', cmap='Reds',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('InceptionV3 - Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    
    plt.tight_layout()
    plt.show()

# Plot confusion matrices
plot_confusion_matrices(eval_results, class_names)

## 9. Final Comparison

In [ ]:
# Final comparison summary
def generate_final_summary(eval_results):
    print("\n" + "="*60)
    print("FINAL MODEL COMPARISON")
    print("="*60)
    
    resnet_metrics = eval_results['resnet_metrics']
    inception_metrics = eval_results['inception_metrics']
    
    print("\nTest Set Performance:")
    print("-" * 50)
    print(f"{'Metric':<15} {'ResNet50':<12} {'InceptionV3':<12} {'Winner'}")
    print("-" * 50)
    
    metrics_to_compare = ['accuracy', 'precision', 'recall', 'f1_score']
    resnet_wins = 0
    inception_wins = 0
    
    for metric in metrics_to_compare:
        if metric in resnet_metrics and metric in inception_metrics:
            resnet_val = resnet_metrics[metric]
            inception_val = inception_metrics[metric]
            
            if resnet_val > inception_val:
                winner = "ResNet50"
                resnet_wins += 1
            elif inception_val > resnet_val:
                winner = "InceptionV3"
                inception_wins += 1
            else:
                winner = "Tie"
            
            print(f"{metric.capitalize():<15} {resnet_val:.4f}      {inception_val:.4f}      {winner}")
    
    print("-" * 50)
    print(f"Overall Winner: {'ResNet50' if resnet_wins > inception_wins else 'InceptionV3' if inception_wins > resnet_wins else 'Tie'}")
    print(f"Score: ResNet50: {resnet_wins}, InceptionV3: {inception_wins}")
    
    # Model characteristics
    print("\nModel Characteristics:")
    print("-" * 30)
    print(f"ResNet50 parameters: {resnet_model.count_params():,}")
    print(f"InceptionV3 parameters: {inception_model.count_params():,}")
    
    return {
        'resnet_metrics': resnet_metrics,
        'inception_metrics': inception_metrics,
        'resnet_wins': resnet_wins,
        'inception_wins': inception_wins
    }

# Generate final summary
final_summary = generate_final_summary(eval_results)

print("\n✅ Model comparison completed!")
print(f"📁 Models saved in: {PROJECT_DIR}")

## 10. Save Results

In [ ]:
# Save all results for future use
def save_results():
    results_dir = os.path.join(PROJECT_DIR, 'results')
    os.makedirs(results_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print("Saving results...")
    
    # Save models
    resnet_model.save(os.path.join(results_dir, f"resnet50_{timestamp}.keras"))
    inception_model.save(os.path.join(results_dir, f"inceptionv3_{timestamp}.keras"))
    
    # Save evaluation results
    with open(os.path.join(results_dir, f"eval_results_{timestamp}.pkl"), 'wb') as f:
        pickle.dump(eval_results, f)
    
    # Save summary
    summary_data = {
        'timestamp': timestamp,
        'final_summary': {k: (v.tolist() if isinstance(v, np.ndarray) else float(v) if isinstance(v, (np.float32, np.float64)) else v) for k, v in final_summary.items()},
        'class_names': class_names,
        'class_weights': {str(k): float(v) for k, v in class_weights.items()}
    }
    
    with open(os.path.join(results_dir, f"summary_{timestamp}.json"), 'w') as f:
        json.dump(summary_data, f, indent=2)
    
    print(f"✅ Results saved with timestamp: {timestamp}")
    return timestamp

# Save results
save_timestamp = save_results()

## 11. Load Results

In [ ]:
# Load previously saved results
def load_results(timestamp=None):
    results_dir = os.path.join(PROJECT_DIR, 'results')
    
    if timestamp is None:
        # Find latest timestamp
        files = [f for f in os.listdir(results_dir) if f.startswith('summary_')]
        if not files:
            print("No saved results found")
            return False
        timestamp = max([f.split('_')[1].split('.')[0] for f in files])
    
    print(f"Loading results from: {timestamp}")
    
    try:
        # Load models
        global resnet_model, inception_model, eval_results, final_summary
        
        resnet_model = keras.models.load_model(os.path.join(results_dir, f"resnet50_{timestamp}.keras"))
        inception_model = keras.models.load_model(os.path.join(results_dir, f"inceptionv3_{timestamp}.keras"))
        
        # Load evaluation results
        with open(os.path.join(results_dir, f"eval_results_{timestamp}.pkl"), 'rb') as f:
            eval_results = pickle.load(f)
        
        # Load summary
        with open(os.path.join(results_dir, f"summary_{timestamp}.json"), 'r') as f:
            data = json.load(f)
            final_summary = data['final_summary']
        
        print("✅ Results loaded successfully!")
        return True
        
    except Exception as e:
        print(f"❌ Error loading results: {e}")
        return False

# Quick load function
# load_results()  # Uncomment to load latest results

## 12. Gradio Interface

In [ ]:
# Install and import Gradio
try:
    import gradio as gr
    import plotly.graph_objects as go
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio", "plotly"])
    import gradio as gr
    import plotly.graph_objects as go

from PIL import Image, ImageEnhance
import cv2

print("Gradio interface ready!")

In [ ]:
# Gradio helper functions
def preprocess_for_model(image, model_type='resnet50'):
    if isinstance(image, Image.Image):
        image = np.array(image)
    
    if len(image.shape) == 3 and image.shape[2] == 4:
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
    elif len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    
    image = cv2.resize(image, IMG_SIZE)
    image = image.astype(np.float32)
    
    if image.max() <= 1.0:
        image = image * 255.0
    
    image = np.expand_dims(image, axis=0)
    
    if model_type == 'resnet50':
        image = resnet_preprocess(image)
    else:
        image = inception_preprocess(image)
    
    return image

def create_confidence_plot(predictions, model_name):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=class_names,
        y=predictions * 100,
        text=[f'{p:.1f}%' for p in predictions * 100],
        textposition='auto'
    ))
    
    fig.update_layout(
        title=f'{model_name} Confidence Scores',
        xaxis_title='Disease Class',
        yaxis_title='Confidence (%)',
        yaxis=dict(range=[0, 100]),
        height=400
    )
    
    return fig

def predict_disease(image):
    if image is None:
        return "Please upload an image", None, None, None
    
    try:
        # Preprocess for both models
        resnet_img = preprocess_for_model(image, 'resnet50')
        inception_img = preprocess_for_model(image, 'inceptionv3')
        
        # Predictions
        resnet_pred = resnet_model.predict(resnet_img, verbose=0)[0]
        inception_pred = inception_model.predict(inception_img, verbose=0)[0]
        
        # Top predictions
        resnet_top = np.argmax(resnet_pred)
        inception_top = np.argmax(inception_pred)
        
        # Format results
        results = f"""
## ResNet50 Prediction
**Primary:** {class_names[resnet_top]} ({resnet_pred[resnet_top]*100:.1f}%)

## InceptionV3 Prediction  
**Primary:** {class_names[inception_top]} ({inception_pred[inception_top]*100:.1f}%)

## Model Agreement
{'✅ Both models agree' if resnet_top == inception_top else '⚠️ Models disagree'}
"""
        
        # Create plots
        resnet_plot = create_confidence_plot(resnet_pred, "ResNet50")
        inception_plot = create_confidence_plot(inception_pred, "InceptionV3")
        
        # Summary table
        summary_df = pd.DataFrame({
            'Model': ['ResNet50', 'InceptionV3'],
            'Prediction': [class_names[resnet_top], class_names[inception_top]],
            'Confidence': [f'{resnet_pred[resnet_top]*100:.1f}%', f'{inception_pred[inception_top]*100:.1f}%']
        })
        
        return results, resnet_plot, inception_plot, summary_df
        
    except Exception as e:
        return f"Error: {str(e)}", None, None, None

print("Prediction functions ready!")

In [ ]:
# Create Gradio interface
def create_interface():
    with gr.Blocks(title="OCTDL Eye Disease Classification", theme=gr.themes.Soft()) as interface:
        
        gr.HTML("""
        <div style="text-align: center; padding: 2rem; background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 10px;">
            <h1>🏥 OCTDL Eye Disease Classification</h1>
            <h3>ResNet50 vs InceptionV3 Model Comparison</h3>
            <p>Advanced Machine Learning Final Project | June 2025</p>
        </div>
        """)
        
        with gr.Row():
            with gr.Column():
                gr.Markdown("### Upload OCT Image")
                image_input = gr.Image(type="pil", label="OCT Image", height=300)
                predict_btn = gr.Button("🔍 Analyze Image", variant="primary", size="lg")
                
                gr.Markdown("""
                **Supported:** JPG, PNG, TIFF  
                **Classes:** AMD, DME, ERM, NO, RAO, RVO, VID  
                **Note:** For research purposes only
                """)
        
        gr.Markdown("## 📊 Analysis Results")
        
        results_text = gr.Markdown(value="Upload an image to see predictions")
        
        with gr.Row():
            summary_table = gr.Dataframe(
                headers=["Model", "Prediction", "Confidence"],
                label="Summary"
            )
        
        with gr.Row():
            resnet_plot = gr.Plot(label="ResNet50 Confidence")
            inception_plot = gr.Plot(label="InceptionV3 Confidence")
        
        # Connect events
        predict_btn.click(
            fn=predict_disease,
            inputs=[image_input],
            outputs=[results_text, resnet_plot, inception_plot, summary_table]
        )
        
        image_input.change(
            fn=predict_disease,
            inputs=[image_input],
            outputs=[results_text, resnet_plot, inception_plot, summary_table]
        )
        
        gr.HTML("""
        <div style="text-align: center; padding: 1rem; margin-top: 2rem; border-top: 1px solid #ddd; color: #666;">
            <p><strong>⚠️ Disclaimer:</strong> For research purposes only - Not for clinical diagnosis</p>
        </div>
        """)
    
    return interface

# Create interface
if 'resnet_model' in globals() and 'inception_model' in globals():
    gradio_interface = create_interface()
    print("✅ Gradio interface created!")
else:
    print("⚠️ Models not loaded. Run training or load_results() first.")

In [ ]:
# Launch Gradio interface
def launch_interface():
    if 'gradio_interface' not in globals():
        print("❌ Interface not created. Run the previous cell first.")
        return
    
    print("🚀 Launching interface...")
    print("🌐 Interface will open at: http://localhost:7860")
    
    gradio_interface.launch(
        server_name="127.0.0.1",
        server_port=7860,
        share=False,
        inbrowser=True,
        quiet=False
    )

# Instructions
print("🎯 Ready to launch!")
print("Run: launch_interface()")

---
**Project Complete** ✅  
All models trained, evaluated, and ready for deployment via Gradio interface.